# The script cleans up and normalizes the addresses and contact information

##1. Load and clean up data

In [5]:
import pandas as pd
import numpy as np

def normalizeStreetName(row, addr_lookup):
    в = process.extractOne(row['Адрес'], choices=addr_lookup, score_cutoff=70)
    if abbrev:
        return state_to_code[abbrev[0]]
    return 'error'

def normalizeStreetType(row):
     
    if abbrev:
        return state_to_code[abbrev[0]]
    return 'error'


raw_data = pd.read_excel('sample.xlsx', sheetname='DB', header=2, encoding='utf-8').reset_index(drop=True)
raw_data.Адрес = raw_data.Адрес.replace(to_replace='(П|п)(Р|р)\.', value='просп. ', regex=True)


addr_lookup = pd.read_excel('sample.xlsx', sheetname='cat', header=1, encoding='utf-8')
addr_lookup = addr_lookup.reset_index(drop=True).dropna(axis=1, how='all')

raw_data.dropna(axis=0, how='all', inplace=True)
raw_data.drop(raw_data.columns[4:], axis=1, inplace=True)
# raw_data.dropna(axis=1, how='all', inplace=True)

#swap address and tel if misplaced
swap_idx = raw_data.Адрес.str.contains(r'^[\d\s-]+$', na=True) #index for rows to swap address and tel 
raw_data.loc[swap_idx, ['Адрес', 'Телефон 1']] = raw_data[swap_idx][['Телефон 1','Адрес']].values
raw_data.Адрес.fillna(value='<no address>', inplace=True)
raw_data[raw_data.Адрес.str.contains('<no address>')]

ukr_addr = pd.read_table('streets_ukr_for_data_preprocessed.txt', header=None, names=['Адрес'])
rus_addr = pd.read_table('streets_rus_for_data_preprocessed.txt', header=None, names=['Адрес'])
raw_data['rus_addr'] = rus_addr
raw_data.rus_addr.replace(to_replace='(П|п)роспект', value='просп.', regex=True, inplace=True)
raw_data.rus_addr.replace(to_replace='(П|п)-т', value='просп. ', regex=True, inplace=True)
raw_data.rus_addr.replace(to_replace='^(В|в)ул(\.|\s)', value='ул. ', regex=True, inplace=True)
raw_data.rus_addr.replace(to_replace='^(В|в)(\.|\s)', value='ул. ', regex=True, inplace=True)
raw_data.rus_addr.replace(to_replace='^(Н|н)(\.|\s)?(аб)?.*Перемог.', value='ул. Набережная Победы', regex=True, inplace=True)
# raw_data.Адрес[~raw_data.Адрес.isin(ukr_addr.Адрес)]
# len(ukr_addr), len(raw_data), len(rus_addr)
raw_data.head()

OSError: zipimport: can not open file C:\programs\development\Anaconda3 2.1.0\lib\site-packages\setuptools-17.1.1-py3.4.egg

## Split street name and address

In [474]:
split_topol_addr_regex = r'^(\D*|.+\d)(?=(?<=Тополь 1|Т-1|Т-2|Т-3|Т-4)\D*\d)(.*)$'
split_topol_addr_regex_named = r'^(?P<Street>\D*|.+\d)(?=(?<=Тополь 1|Т-1|Т-2|Т-3|Т-4)\D*\d)(?P<addr>.*)$'

split_addr_regex = r'^(\D*|.+\d)(?=\D*\d)(.*)$'
split_addr_regex_named=r'^(?P<street>\D*)(?P<addr>.*)$'

raw_data['street'], raw_data['addr'] = zip(*raw_data.rus_addr.str.extract(split_addr_regex_named).values)
raw_data.head(10)


,Качество,Адрес,Телефон 1,Телефон 2,rus_addr,street,addr
0,плохое,"вул. Дємєнтьєва, 2/71",066-574-01-31,NaN,"ул. Дементьева, 2/71","ул. Дементьева,",2/71
1,плохое,"Вул.Железнодорожна,15",0,NaN,"ул. Железнодорожна, 15","ул. Железнодорожна,",15
2,плохое,"ул.Гагарина, 37 кв.4 (Першотравенск)",502240878,NaN,"ул.Гагарина, 37 кв.4 (Першотравенск)","ул.Гагарина,",37 кв.4 (Першотравенск)
3,плохое,"просп. Гагарина, д37 кв.4 Першотравенск",502240878,NaN,"просп. Гагарина, Д37 кв.4 Першотравенск","просп. Гагарина, Д",37 кв.4 Першотравенск
4,плохое,Наб. Перемоги 130/1/189,675688508,NaN,Наб. Победы 130/1/189,Наб. Победы,130/1/189
5,плохое,в.Красночечелівська//б.46 кв.22,500514364,NaN,ул. Красночечеливська // б.46 кв.22,ул. Красночечеливська // б.,46 кв.22
6,плохое,"п-т Ілліча, буд. 21, кв. 75",0960414961 0960414963,NaN,"просп. Ильича, д. 21 кв. 75","просп. Ильича, д.",21 кв. 75
7,плохое,ул.Володарского 109а,501926152,NaN,ул.Володарского 109а,ул.Володарского,109а
8,плохое,"Запорізьке шосе, б.40, кв.357",994695916; 0666850000,NaN,"Запорожское шоссе, б.40, кв.357","Запорожское шоссе, б.","40, кв.357"
9,плохое,"просп. Миру, 71/62",097-361-00-41,NaN,"просп. Мира, 71/62","просп. Мира,",71/62


## Translate street names (don't abuse, might be Google-banned )

In [ ]:
import goslate
from urllib import request


proxy_handler = request.ProxyHandler({"http" : "http://176.31.119.64:8080"})
proxy_opener = request.build_opener(request.HTTPHandler(proxy_handler), request.HTTPSHandler(proxy_handler))
gs = goslate.Goslate(opener=proxy_opener)
streets = [str(x) for x in raw_data.Адрес.tolist()]
streets_rus = gs.translate(streets, 'ru', source_language='uk')
translation = list(streets_rus)

##Lookup and match street names

In [4]:
import multiprocessing
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time

df = raw_data.head(n = 10)

street_choices = [str(x) for x in addr_lookup.Улица.dropna().tolist()]

start = time.clock()
matched_streets = list()
for raw in df.street.tolist():    
    scores = [(fuzz.token_set_ratio(raw, normal), normal) for normal in street_choices]
    matched_streets.append(max(scores, key=lambda x: x[0])[1])
#     match = process.extractOne(raw, street_choices)
#     matched_streets.append(match[0])
    
# print(matched_streets)
# for w in choices:
#     print(w)
#     print(fuzz.partial_token_sort_ratio('вул. Дємєнтьєва, 2/71', w))
print('time: ' + str(time.clock() - start))

for (x,y) in zip(df.street.tolist(), matched_streets):
    print (x,y)
# pd.isnull(addr_lookup['Unnamed: 1'])
# addr_lookup['Unnamed: 1'].iloc[1]

# df = raw_data.dropna(how='all')
# # df = raw_data.drop(raw_data.index[1])
# df = df[df.Адрес.str.contains(u'Правд')==True]
# df



choices = ['просп. Воронцова', 'просп. Гагарина', 'просп. Газеты "Правда"', 'просп. Героев', 'просп. Ильича', 
           'просп. Карла Маркса', 'просп. Кирова', 'просп. Металлургов', 'просп. Мира', 'просп. Олимпийский',
           'просп. Петровского', 'просп. Пушкина', 'просп. Свободы', 'просп. Сергея Нигояна', 'просп. Труда',
           'ул. Правды', 'ул. Юдина', 'ул. Набережная',  'ул. Набережная В.И.Ленина', 'пер. Пролетарской Победы', 
           'ул. Набережная Заводская', 'ул. Набережная им. Ленина', 'ул. Набережная Победы', 
           'ул. Мира', 'ул. Железнодорожная', 'ул. Дорожная', 'пер. Пролетарской Победы']    

process.extract(u'ул. Железнодорожна', choices)
fuzz.partial_token_sort_ratio(u'Наб. Победы', 'ул. Набережная Победы')
fuzz.partial_ratio(u'Наб. Победы', 'ул. Набережная Победы')


sorted([(fuzz.token_set_ratio('20-річчя Победы,', normal), normal) for normal in street_choices], reverse=True)

from multiprocessing import Pool
p = Pool(5)
def f(x):
     return x*x

p.map(f, [1,2,3])


C:\programs\development\Anaconda3 2.1.0\lib\site-packages\fuzzywuzzy\fuzz.py:33: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


NameError: name 'raw_data' is not defined

In [ ]:
translation

In [314]:
import heapq
choices = addr_lookup.Улица.tolist()
choices = [str(x) for x in choices]              
# choices = process.extract("проспект Правди", choices, limit=2)
# print(choices)    
# choices = ['просп. Воронцова', 'просп. Гагарина', 'просп. Газеты "Правда"', 'просп. Героев', 'просп. Ильича', 
#            'просп. Карла Маркса', 'просп. Кирова', 'просп. Металлургов', 'просп. Мира', 'просп. Олимпийский',
#            'просп. Петровского', 'просп. Пушкина', 'просп. Свободы', 'просп. Сергея Нигояна', 'просп. Труда',
#            'ул. Правды']    
process.extract(u'Т-1', choices)
# items = [(fuzz.partial_token_sort_ratio('проспект Правди', w), w) for w in choices]
# heapq.nlargest(3, items, key=lambda item: item[0])

    

[('ж/м Тополь-1', 85),
 ('пер. 1-й Наклонный', 85),
 ('пер. Веснина 1-й', 85),
 ('тупик Одоевского 1-й', 85),
 ('тупик Рыбинский 1-й', 85)]

In [2]:
from numbapro import vectorize, 
os.path.dirname(pycuda.__file__)

ImportError: No module named 'pycuda'